In [1]:
from math import log
import operator

In [2]:
#计算给定数据集的香农熵
def calcShannonEnt(dataSet):
    #计算数据集中的实例总数
    numEntries=len(dataSet)
    #分类字典（key:分类名称 value:数量）
    labelCounts={}
    #遍历数据集，将分类名称，数量添加到分类字典中
    for featVec in dataSet:
        currentLabel=featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel]=0
        labelCounts[currentLabel]+=1
    shannonEnt=0.0
    #计算所有类别的期望值总和
    for key in labelCounts:
        #类别发生频率
        prob=float(labelCounts[key])/numEntries
        shannonEnt-=prob*log(prob,2)
    return shannonEnt

In [3]:
#创建数据集
def createDataSet():
    dataSet=[[1,1,'yes'],[1,1,'yes'],[1,0,'no'],[0,1,'no'],[0,1,'no']]
    labels=['no surfacing','flippers']
    return dataSet,labels

In [4]:
#按照给定特征划分数据集(dataSet:待划分数据集 axis:划分数据集特征 value:特征返回值)
def splitDataSet(dataSet,axis,value):
    #创建新的列表对象，用于保存划分后的数据集
    retDataSet=[]
    #遍历数据集中的每个元素，一旦发现符合要求的值，将其添加到新创建的列表中
    for featVec in dataSet:
        #将符合特征的数据抽取出来
        if featVec[axis]==value:
            reducedFeatVec=featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [5]:
#选择最好的数据集划分方式
def chooseBestFeatureToSplit(dataSet):
        #当前数据集包含的特征属性数量
        numFeatures=len(dataSet[0])-1
        #计算整个数据集的香农熵
        baseEntropy=calcShannonEnt(dataSet)
        bestInfoGain=0.0;bestFeature=-1
        #遍历数据集中的所有特征值
        for i in range(numFeatures):
            #将数据集中所有第i个特征值或者可能存在的值写入列表
            featList=[example[i] for example in dataSet]
            #使用列表创建集合（可过滤掉重复的元素）
            uniqueVals=set(featList)
            newEntropy=0.0
            for value in uniqueVals:
                subDataSet=splitDataSet(dataSet,i,value)
                prob=len(subDataSet)/float(len(dataSet))
                newEntropy+=prob*calcShannonEnt(subDataSet)
            infoGain=baseEntropy-newEntropy
            if (infoGain>bestInfoGain):
                bestInfoGain=infoGain
                bestFeature=i
        return bestFeature

In [6]:
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys():classCount[vote]=0
        classCount[vote]+=1
    shortedClassCount=shorted(classCount.iteriyems(),key=operator.itemgetter(1),reverse=True)
    return shortedClassCount[0][0]

In [7]:
#创建决策树
def createTree(dataSet,labels):
    classList=[example[-1] for example in dataSet]
    if classList.count(classList[0])==len(classList):
        return classList[0]
    if len(dataSet[0])==1:
        return majorityCnt(classList)
    bestFeat=chooseBestFeatureToSplit(dataSet)
    bestFeatLabel=labels[bestFeat]
    myTree={bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues=[example[bestFeat] for example in dataSet]
    uniqueVals=set(featValues)
    for value in uniqueVals:
        subLabels=labels[:]
        myTree[bestFeatLabel][value]=createTree(splitDataSet(dataSet,bestFeat,value),subLabels)
    return myTree